In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("DataExplore") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

# VERİ SETİNİ OKUMA ÖNCESİ KONTROLLER
1. Veri setine ulaşma:
    adult.data ve adult.test adında iki veri dosyası var.
2. Mümkünse Notepad++ ile içine bakalım. Baktık. virgülle ayrılmış ve başlıkları olan bir dosya

In [3]:
adult_train_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("D:\\Datasets\\adult.data") \

In [4]:
adult_test_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("D:\\Datasets\\adult.test") \

In [5]:
adult_train_df.show(5)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|output|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|       

DF çok geniş olduğu için düzgün bir görünüm alamadık. Driver'a aldığımız birkaç satırdan oluşan dataframe'i pandas dataframe'e çevirerek daha güzel bir görünüm sağlayabiliriz.

In [6]:
adult_train_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [7]:
adult_test_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,output
0,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
1,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
2,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
3,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
4,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.


In [8]:
adult_whole_df = adult_train_df.union(adult_test_df)
adult_whole_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [9]:
print("train satır sayısı: ", adult_train_df.count())
print("test satır sayısı:  ", adult_test_df.count())
print("whole satır sayısı: ", adult_whole_df.count())

train satır sayısı:  32561
test satır sayısı:   16281
whole satır sayısı:  48842


Satır sayıları tutarlı olduğu için başlık yanlışlıkla whale_df içine girmemiş. Aferin union'a.

# VERİ SETİNİ İNCELEME ŞEMA İLE KARŞILAŞTIRMA

In [10]:
adult_whole_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- output: string (nullable = true)



# NÜMERİK DEĞİŞKENLERİN İSTATİSTİKLERİ

In [11]:
adult_whole_df.describe(['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']) \
.toPandas().head()

,summary,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,count,48842,48842,48842,48842,48842,48842
1,mean,38.64358543876172,189664.13459727284,10.078088530363212,1079.0676262233324,87.50231358257237,40.422382375824085
2,stddev,13.710509934443563,105604.02542315733,2.570972755592263,7452.019057655401,403.0045521243599,12.3914440242523
3,min,17,12285.0,1.0,0.0,0.0,1.0
4,max,90,1490400.0,16.0,99999.0,4356.0,99.0


# KATEGORİK DEĞİŞKENLERİN İNCELENMESİ

Kategorik değişkenlerin incelenmesinde groupBy() kullanmak daha çok bilgi verir. Hedef değişken dahil toplam 9 kategorik değişken var.

# 1. workclass

In [12]:
from pyspark.sql import functions as F

In [17]:
adult_whole_df.groupBy(F.col("workclass")) \
.agg({"*":"count"}) \
.toPandas().head(10)

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432
2,Self-emp-not-inc,3862
3,Local-gov,3136
4,Private,33906
5,?,2799
6,Self-emp-inc,1695
7,Without-pay,21
8,Never-worked,10


In [15]:
adult_whole_df.createOrReplaceTempView("workclass_tbl")

In [18]:
spark.sql(" SELECT workclass, COUNT(*) FROM workclass_tbl GROUP BY workclass LIMIT 10") \
.toPandas().head(10)

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432
2,Self-emp-not-inc,3862
3,Local-gov,3136
4,Private,33906
5,?,2799
6,Self-emp-inc,1695
7,Without-pay,21
8,Never-worked,10


Yorum: 2.799 adet ? var. Bu nedir. Muhtelemen kayıp bilgi.
Daha sonra never-worked ve without-pay sınıfları çok az tekrarlanmış. Bunların da veri setinden çıkarılması düşünebilir

# 2. education

In [20]:
adult_whole_df.groupBy(F.col("education")) \
.agg({"*":"count"}) \
.toPandas().head(20)

,education,count(1)
0,Prof-school,834
1,10th,1389
2,7th-8th,955
3,5th-6th,509
4,Assoc-acdm,1601
5,Assoc-voc,2061
6,Masters,2657
7,12th,657
8,Preschool,83
9,9th,756


Yorum: Genel bir sıkıntı görünmüyor ancak çok fazla kategori var belki bazıları birleştirilebilir.

      1st-4th, 5th-6th, 7th-8th: elementary-school

      9th, 10th, 11th, 12th: high-school

      Masters, Doctorate: high-education

      Bachelors, Some-college: undergraduate

# 3. marital_status¶

In [21]:
adult_whole_df.groupBy(F.col("marital_status")) \
.agg({"*":"count"}) \
.toPandas().head(20)

,marital_status,count(1)
0,Widowed,1518
1,Married-spouse-absent,628
2,Married-AF-spouse,37
3,Married-civ-spouse,22379
4,Divorced,6633
5,Never-married,16117
6,Separated,1530


Yorum: Sorun görünmüyor.

# 4. occupation

In [22]:
adult_whole_df.groupBy(F.col("occupation")) \
.agg({"*":"count"}) \
.toPandas().head(20)

,occupation,count(1)
0,Farming-fishing,1490
1,Handlers-cleaners,2072
2,Prof-specialty,6172
3,Adm-clerical,5611
4,Exec-managerial,6086
5,Craft-repair,6112
6,Sales,5504
7,?,2809
8,Tech-support,1446
9,Transport-moving,2355


Yorum: 2809 tane ? var. Bunlar muhtemelen bilinmeyenler. Ayrıca Armed-Forces 15 kişi. Bu sınıfa ait kayıtlar çıkarılabilir.

# 5. relationship

In [24]:
adult_whole_df.groupBy(F.col("relationship")) \
.agg({"*":"count"}) \
.toPandas().head(20)

,relationship,count(1)
0,Husband,19716
1,Own-child,7581
2,Not-in-family,12583
3,Other-relative,1506
4,Wife,2331
5,Unmarried,5125


Yorum: Sorun yok

# 6. race

In [25]:
adult_whole_df.groupBy(F.col("race")) \
.agg({"*":"count"}) \
.toPandas().head(20)

,race,count(1)
0,Asian-Pac-Islander,1519
1,Black,4685
2,Other,406
3,White,41762
4,Amer-Indian-Eskimo,470


Yorum: Sorun görünmüyor. Çoğunluk white.

# 7. sex

In [26]:
adult_whole_df.groupBy(F.col("sex")) \
.agg({"*":"count"}) \
.toPandas().head(20)

,sex,count(1)
0,Male,32650
1,Female,16192


Yorum: Sorun görünmüyor

# 8. native_country

In [27]:
adult_whole_df.groupBy(F.col("native_country")) \
.agg({"*":"count"}) \
.toPandas().head(50)

,native_country,count(1)
0,Dominican-Republic,103
1,Ireland,37
2,Cuba,138
3,Guatemala,88
4,Iran,59
5,Taiwan,65
6,El-Salvador,155
7,United-States,43832
8,South,115
9,Japan,92


Holand-Netherlands 1, büyük çoğunluk USA'den

# 9. output

In [29]:
adult_whole_df.groupBy(F.col("output")) \
.agg({"*":"count"}) \
.show()

+-------+--------+
| output|count(1)|
+-------+--------+
|   >50K|    7841|
|  >50K.|    3846|
| <=50K.|   12435|
|  <=50K|   24720|
+-------+--------+



Yorum: "." içeren sonuçlar var. Bunların temizlenmesi gerekir.

/////////////////////// VERİ TEMİZLİĞİ İÇİN TAVSİYELER ////////////////////////////////////////////////

  1. Tüm sütunları boşluk kontrolü yap.
  2. ? içeren workclass, occupation  var bunların ? içerdiği satırlar tekrar incelenmeli.
      ? işaretleri sistematik bir şekilde mi oluşmuş yoksa bu oluşum tesadüfi mi?
      ? kayıtlarının oluşması altında yatan bir mekanizma var mı?
      Bu sistematik hata yakalanırsa veri doldurma (imputation) yoksa satır silme yapılsın.
  3. workclass niteliğinde never-worked ve without-pay sınıfları ve
      occupation niteliğinde  Armed-Forces  sınıfı
    çok az tekrarlanmış. Veri setinden çıkarılabilir.
  4. education niteliğindeki:
          1st-4th, 5th-6th, 7th-8th: elementary-school
          9th, 10th, 11th, 12th: high-school
          Masters, Doctorate: high-education
          Bachelors, Some-college: undergraduate
     sınıfları yukarıdaki gibi birleştirilebilir.
  5. native country'de ? var ve Hollanda sadece 1 kez tekrarlanmış. 
  6. output (hedef değişkendeki) >50K. ve <=50K. değerlerindeki "." kaldırılmalı